<a href="https://colab.research.google.com/github/and749/Projeto-de-imers-o-para-o-certificado/blob/main/Chat_bot_inicial_integra%C3%A7%C3%A3o_WhastApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o SDK do google

In [ ]:
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('secret_key')
GOOGLE_API_KEY="secret_key"
genai.configure(api_key=api_key)


Listar os modelos disponíveis

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5
}

In [ ]:
safety_settings = {
	"HARASSMENT": "BLOCK_NONE" ,
	"HATE" : "BLOCK_NONE" ,
	"SEXUAL": "BLOCK_NONE" ,
	"DANGEROUS": "BLOCK_NONE",
}

Inicializando modelo

In [ ]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro" ,
generation_config=generation_config,
safety_settings=safety_settings)

In [ ]:
response = model.generate_content("vamos aprender conteúdo sobre IA, me de sugestões")
print(response.text)

**Recursos Online:**

* **Coursera:** Cursos sobre IA, aprendizado de máquina e aprendizado profundo
* **edX:** Cursos sobre IA, ciência de dados e engenharia de software
* **MIT OpenCourseWare:** Cursos sobre IA, aprendizado de máquina e processamento de linguagem natural
* **Google AI Education:** Tutoriais, cursos e recursos sobre IA e aprendizado de máquina
* **Microsoft Learn:** Cursos e módulos sobre IA, aprendizado de máquina e ciência de dados

**Livros:**

* **Inteligência Artificial: Uma Abordagem Moderna (3ª edição)** por Stuart Russell e Peter Norvig
* **Aprendizado de Máquina (4ª edição)** por Ethem Alpaydin
* **Aprendizado Profundo** por Ian Goodfellow, Yoshua Bengio e Aaron Courville
* **IA para Todos** por Andrew Ng
* **Inteligência Artificial: Uma Introdução** por Melanie Mitchell

**Revistas e Publicações:**

* **Nature Machine Intelligence:** Revista científica sobre IA e aprendizado de máquina
* **IEEE Transactions on Neural Networks and Learning Systems:** Publicaç

In [ ]:
chat = model.start_chat(history=[])

In [ ]:
prompt = input("esperando prompt: ")

while prompt != "fim" :
  response = chat.send_message(prompt)
  print("resposta " , response.text, "\n")
  prompt = input("esperando prompt: ")

esperando prompt: qual foi a primeira noticia que vimos hoje?
resposta  Como um assistente de IA, não tenho acesso a informações em tempo real e, portanto, não posso fornecer a primeira notícia que você viu hoje. 

esperando prompt: Qual foi o primeiro titulo de noticia que eu te encaminhei
resposta  Como um assistente de IA, não tenho acesso a informações pessoais ou mensagens privadas, incluindo títulos de notícias que você possa ter me encaminhado. 

esperando prompt: qual a capital do japão?
resposta  Tóquio 

esperando prompt: fim


In [ ]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')

> **user**: qual foi a primeira noticia que vimos hoje?

-------------------------------------------


> **model**: Como um assistente de IA, não tenho acesso a informações em tempo real e, portanto, não posso fornecer a primeira notícia que você viu hoje.

-------------------------------------------


> **user**: Qual foi o primeiro titulo de noticia que eu te encaminhei

...

-------------------------------------------


> **user**: qual a capital do japão?

-------------------------------------------


> **model**: Tóquio

-------------------------------------------


In [ ]:
!pip install -q -U google-generativeai twilio flask

import os
import google.generativeai as genai
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse

# Configurar a API do Google Generative AI
GOOGLE_API_KEY = "SUA_API_KEY" # **SUBSTITUA pela sua chave API**
genai.configure(api_key=GOOGLE_API_KEY)

# Configurar as credenciais da Twilio (variáveis de ambiente)
os.environ["TWILIO_ACCOUNT_SID"] = "ACxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # **SUBSTITUA pelo seu Account SID**
os.environ["TWILIO_AUTH_TOKEN"] = "your_auth_token" # **SUBSTITUA pelo seu Auth Token**

# Carregar as credenciais da Twilio das variáveis de ambiente
TWILIO_ACCOUNT_SID = os.environ.get("TWILIO_ACCOUNT_SID")
TWILIO_AUTH_TOKEN = os.environ.get("TWILIO_AUTH_TOKEN")

# Criar o aplicativo Flask
app = Flask(__name__)

# Configurar o modelo Gemini
model = genai.GenerativeModel(
    model_name="gemini-1.0-pro",
    generation_config={"candidate_count": 1, "temperature": 0.5},
    safety_settings={
        "HARASSMENT": "BLOCK_NONE",
        "HATE": "BLOCK_NONE",
        "SEXUAL": "BLOCK_NONE",
        "DANGEROUS": "BLOCK_NONE",
    },
)
chat = model.start_chat(history=[])

# Definir a rota do webhook
@app.route("/webhook", methods=["POST"])
def webhook():
    """Endpoint para receber mensagens do WhatsApp e enviar respostas."""
    incoming_msg = request.values.get("Body", "").strip()

    # Processar a mensagem com o Gemini
    response = chat.send_message(incoming_msg)
    reply = response.text

    # Criar a resposta para Twilio (TwiML)
    resp = MessagingResponse()
    resp.message(reply)

    return str(resp)

# Executar o aplicativo Flask (apenas para testes locais)
if __name__ == "__main__":
    app.run(debug=True)